In [903]:
import cv2
import skimage.io as io
import numpy as np
from skimage.morphology import binary_erosion, binary_dilation, binary_closing,skeletonize, thin
from scipy.signal import convolve2d
from statistics import mode,variance
from math import sqrt
from skimage.measure import find_contours
import matplotlib.pyplot as plt 
from skimage import data, color, img_as_ubyte 
from skimage.feature import canny 
from skimage.transform import hough_ellipse 
from skimage.draw import ellipse_perimeter 
from skimage.draw import rectangle
from skimage.morphology import disk
from scipy.spatial.distance import euclidean


In [904]:
def showBinaryImg(img):
    cv2.imshow("Images",img*255)
    cv2.waitKey(0) 
    cv2.destroyAllWindows()
def showImg(img):
    cv2.imshow("Images",img)
    cv2.waitKey(0) 
    cv2.destroyAllWindows()

In [905]:
def opening(img,ST):
    imgEroded=cv2.erode(img,ST)
    return cv2.dilate(imgEroded,ST)
def closing(img,ST):
    imgDilated=cv2.dilate(img,ST)
    return cv2.erode(imgDilated,ST)

In [906]:
import numpy as np

from skimage.util.shape import view_as_windows

def feng_threshold(img, w_size1=15, w_size2=30,
                   k1=0.15, k2=0.01, alpha1=0.1):
    """ Runs the Feng's thresholding algorithm.
    Reference:
    Algorithm proposed in: Meng-Ling Feng and Yap-Peng Tan, “Contrast adaptive
    thresholding of low quality document images”, IEICE Electron. Express,
    Vol. 1, No. 16, pp.501-506, (2004).
    Modifications: Using integral images to compute the local mean and the
    standard deviation
    @param img: The input image. Must be a gray scale image
    @type img: ndarray
    @param w_size1: The size of the primary local window to compute
        each pixel threshold. Should be an odd window
    @type w_size1: int
    @param w_size2: The size of the secondary local window to compute
        the dynamic range standard deviation. Should be an odd window
    @type w_size2: int
    @param k1: Parameter value that lies in the interval [0.15, 0.25].
    @type k1: float
    @param k2: Parameter value that lies in the interval [0.01, 0.05].
    @type k2: float
    @param alpha1: Parameter value that lies in the interval [0.15, 0.25].
    @type alpha1: float
    @return: The estimated local threshold for each pixel
    @rtype: ndarray
    """
    # Obtaining rows and cols
    rows, cols = img.shape
    i_rows, i_cols = rows + 1, cols + 1

    # Computing integral images
    # Leaving first row and column in zero for convenience
    integ = np.zeros((i_rows, i_cols), np.float)
    sqr_integral = np.zeros((i_rows, i_cols), np.float)

    integ[1:, 1:] = np.cumsum(np.cumsum(img.astype(np.float), axis=0), axis=1)
    sqr_img = np.square(img.astype(np.float))
    sqr_integral[1:, 1:] = np.cumsum(np.cumsum(sqr_img, axis=0), axis=1)

    # Defining grid
    x, y = np.meshgrid(np.arange(1, i_cols), np.arange(1, i_rows))

    # Obtaining local coordinates
    hw_size = w_size1 // 2
    x1 = (x - hw_size).clip(1, cols)
    x2 = (x + hw_size).clip(1, cols)
    y1 = (y - hw_size).clip(1, rows)
    y2 = (y + hw_size).clip(1, rows)

    # Obtaining local areas size
    l_size = (y2 - y1 + 1) * (x2 - x1 + 1)

    # Computing sums
    sums = (integ[y2, x2] - integ[y2, x1 - 1] -
            integ[y1 - 1, x2] + integ[y1 - 1, x1 - 1])
    sqr_sums = (sqr_integral[y2, x2] - sqr_integral[y2, x1 - 1] -
                sqr_integral[y1 - 1, x2] + sqr_integral[y1 - 1, x1 - 1])

    # Computing local means
    means = sums / l_size

    # Computing local standard deviation
    stds = np.sqrt(sqr_sums / l_size - np.square(means))

    # Obtaining windows
    padded_img = np.ones((rows + w_size1 - 1, cols + w_size1 - 1)) * np.nan
    padded_img[hw_size: -hw_size, hw_size: -hw_size] = img

    winds = view_as_windows(padded_img, (w_size1, w_size1))

    # Obtaining maximums and minimums
    mins = np.nanmin(winds, axis=(2, 3))

    # Obtaining local coordinates for std range calculations
    hw_size = w_size2 // 2
    x1 = (x - hw_size).clip(1, cols)
    x2 = (x + hw_size).clip(1, cols)
    y1 = (y - hw_size).clip(1, rows)
    y2 = (y + hw_size).clip(1, rows)

    # Obtaining local areas size
    l_size = (y2 - y1 + 2) * (x2 - x1 + 2)

    # Computing sums
    sums = (integ[y2, x2] - integ[y2, x1 - 1] -
            integ[y1 - 1, x2] + integ[y1 - 1, x1 - 1])
    sqr_sums = (sqr_integral[y2, x2] - sqr_integral[y2, x1 - 1] -
                sqr_integral[y1 - 1, x2] + sqr_integral[y1 - 1, x1 - 1])

    # Computing local means2
    means2 = sums / l_size
    # Computing standard deviation range
    std_ranges = np.sqrt(sqr_sums / l_size - np.square(means2))

    # Computing normalized standard deviations and extra alpha parameters
    n_stds = stds / std_ranges
    n_sqr_std = np.square(n_stds)
    alpha2 = k1 * n_sqr_std
    alpha3 = k2 * n_sqr_std

    thresholds = ((1 - alpha1) * means + alpha2 * n_stds
                  * (means - mins) + alpha3 * mins)
    for x in range(img.shape[0]):
        for y in range(img.shape[1]):
            if(img[x][y]<thresholds[x][y]):
                img[x][y]=0
            else:
                img[x][y]=1
    return img


In [907]:
img= cv2.imread("aloha.png",0) 
# cv2.imwrite("test.jpg",img)
# print(img[:,1])
if(np.amax(img)==1):
    # img=img.astype("uint8")
    # cv2.imwrite("test1.jpg",img)
    print("hi")
else:
    img=(img/255)
img=feng_threshold(img)
# cv2.imwrite("test2.png",img*255)
# print(img[:,1])
img2= np.copy(img)
img=1-img


# cv2.imwrite("test3.jpg",img*255)
# print(img)
proj = np.sum(img,1)
#proj=1-proj
maxProjection = np.max(proj) 
w = 500 
result = np.zeros(img.shape) 
result2 = np.zeros(img.shape) 
width=img.shape[1] 
binOrig=np.copy(img) 
# Draw a line for each row 
for row in range(img.shape[0]):
    result2[row,0:int(proj[row])]=1
    if(proj[row]>.8*maxProjection): 
        cv2.line(result, (0,row), (int(proj[row]*(maxProjection/width)),row), (255,255,255), 1) 
        
cv2.imwrite('result2.png', result2*255) 
cv2.imwrite('result.png', result) 
result=result2
result=1-result
print(proj.shape, img.shape)

(573,) (573, 1063)


In [908]:
def filterPeaks(peaksMids,widths):
    newPeaks=[]
    newWidth=[]
    sumW=0
    for i in widths:
        sumW+=i
    avgW=sumW/len(widths)
    #if i dont have flase peaks for clusters then this will result in a problem
    var = variance(widths)
    print("var:",var)
    for i in range(len(peaksMids)):
        if(widths[i]>=avgW or (var<1000 and (avgW-widths[i])<20)):
            newPeaks.append(peaksMids[i])
            newWidth.append(widths[i])
    return newWidth,newPeaks
def imgStaffSegments(img,peaksMids,widths,staffH):
    segments=[]
    staffH=int(staffH)
    #TODO:add double the staff space instead of 5
    for i in range(len(peaksMids)):
        segments.append(img2[peaksMids[i]-int(widths[i]/2)-2*staffH:peaksMids[i]+int(widths[i]/2)+2*staffH,:])
    return segments

In [909]:
def calcStaffPos(rowHist,isSigment):
    n,m=rowHist.shape
    if(isSigment):
        thres=20
    else:
        thres=int(maxProjection*0.6)
    start=end=0
    width=[]
    #rowHist=(rowHist/255).astype("uint8")
    peaksMid=[]
    for i in range(n-1):
        if(rowHist[i][thres]!=rowHist[i+1][thres]):
            if rowHist[i][thres]==1:
                #end of zeros
                start=i
            else:
                #end of ones
                end=i+1
                peaksMid.append(int((start+end)/2))
                width.append((end-start))
    
    return np.array(width),np.array(peaksMid)


width,peaksMids=calcStaffPos(result,False)
maxSpace=0
for i in range(len(peaksMids)):
    if(i%5<4 and i+1<len(peaksMids)and peaksMids[i+1]-peaksMids[i]>maxSpace):
        maxSpace=peaksMids[i+1]-peaksMids[i]
        maxSpace=maxSpace-(width[i]/2+width[i+1]/2)

widthSegments,peakSegments=calcStaffPos(result,True)
print(width)
widthSegments,peakSegments=filterPeaks(peakSegments,widthSegments)
imgStaffSegments=imgStaffSegments(img2,peakSegments,widthSegments,maxSpace)

# print(imgStaffSegments)
imgindex=0
for i in imgStaffSegments:
    cv2.imwrite("segments/"+str(imgindex)+".png",i*255)
    imgindex+=1



[2 2 3 3 3 2 3 3 3 2 2 3 3 2 2 2 2 3 3 2]
var: 1124


In [910]:
def removeStaffRow(imgOriginal,midPoint,curWidth):
    thresPixel=curWidth
    for i in range(imgOriginal.shape[1]):
        pixelSum= sum(imgOriginal[midPoint-curWidth:midPoint+curWidth,i:i+1])
        if(pixelSum<=thresPixel):
           # print(imgOriginal[midPoint-curWidth:midPoint+curWidth,i:i+1])
            imgOriginal[midPoint-curWidth:midPoint+curWidth,i:i+1]=0
    return imgOriginal

img2=1-img2
ST=np.ones((1,3))
img2=cv2.dilate(img2,ST)

for i in range(len(peaksMids)):
    img2=removeStaffRow(img2,peaksMids[i],width[i])

ST=np.ones((2,1))
img2=cv2.dilate(img2,ST)
img2=1-img2
cv2.imwrite("outing.png",img2*255)


True

In [911]:
def detectStaffOrient():
    img_rgb = cv2.imread("outing.png",1)  
    img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)   
    template = cv2.imread("template.png",0) 
    width,height=template.shape
    match = cv2.matchTemplate(img_gray, template, cv2.TM_CCOEFF_NORMED)
    threshold = 0.8
    position = np.where(match >= threshold)
    for point in zip(*position[::-1]): #draw the rectangle around the matched template
        cv2.rectangle(img_rgb, point, (point[0] + width, point[1] + height), (153, 22, 0), 0)
    
    cv2.imwrite("circleDetected.png",img_rgb)

In [912]:
# import cv2
# import numpy as np;

# # Read image
# im = cv2.imread("note.png", cv2.IMREAD_GRAYSCALE)

# # Set up the detector with default parameters.
# im=cv2.bitwise_not(im)

# params = cv2.SimpleBlobDetector_Params()
# params.filterByInertia = False
# params.filterByConvexity = False
# params.filterByCircularity = True
# params.minCircularity = 0.1
# detector = cv2.SimpleBlobDetector_create(params)


# # Detect blobs.
# keypoints = detector.detect(im)
# im=cv2.bitwise_not(im)
# # Draw detected blobs as red circles.
# # cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS ensures the size of the circle corresponds to the size of blob
# im_with_keypoints = cv2.drawKeypoints(im, keypoints, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

# # Show keypoints
# cv2.imshow("Keypoints", im_with_keypoints)
# cv2.waitKey(0)
# cv2.destroyAllWindows()


In [913]:
# from math import sqrt
# from skimage import data
# from skimage.feature import blob_dog, blob_log, blob_doh
# from skimage.color import rgb2gray
# import matplotlib.pyplot as plt
# from collections import deque



# image_gray=img2

# ST1=np.array(list(zip(*ST))[::2])
# ST2=np.array(list(zip(*ST))[::-2])


# IMG1=closing(image_gray,ST1)
# IMG2=closing(IMG1,ST2)

                


# cv2.imshow("1", IMG1) 
# cv2.waitKey(0) 
# cv2.destroyAllWindows()

# cv2.imshow("2", IMG2) 
# cv2.waitKey(0) 
# cv2.destroyAllWindows()



In [914]:
def testContour(contour,index,count,Xmin, Xmax, Ymin, Ymax):
    #center 3la 2l midPeak
    cv2.imwrite("symbols/"+str(index)+str(count)+".png",contour*255)
    aspectRatio=(Xmax - Xmin) / (Ymax - Ymin)
    area=(Xmax - Xmin) * (Ymax - Ymin)
    print(aspectRatio," , ",Xmin," , ", Xmax," , ", Ymin," , ", Ymax," , ",index," , ",count," , ",area)
    if(aspectRatio>1.01 and  aspectRatio < 2.5 and area>30 and area < 130):
        cv2.imwrite("symbolsOut/"+str(index)+str(count)+".png",contour*255)

    #Beams
    #Chords
    #symbol

def getNotesHeads(symbol,index):
    symbol=1-symbol
    ST=disk(.5*maxSpace)
    symbol=symbol.astype("uint8")
    dilated=cv2.dilate(symbol,ST)
    ST=disk(maxSpace*1.5)
    opened=opening(dilated,ST)
    cv2.imwrite("blobs/"+str(index)+".png",opened*255)
    contours = find_contours(opened, 0.8)
    imgContours=[]
    count=0

    
    
    
    

In [915]:
contours = find_contours(img2, 0.8)
imgContours=[]
index=0
for contour in contours:
    x = contour[:,1]
    y = contour[:,0]
    [Xmin, Xmax, Ymin, Ymax] = [np.amin(x), np.amax(x), np.amin(y), np.amax(y)]
    rr, cc = rectangle(start = (Ymin,Xmin), end = (Ymax,Xmax), shape=img.shape)
    imgSymbol=img2[int(Ymin):int(Ymax+1),int(Xmin):int(Xmax+1)]
    #getNotesHeads(imgSymbol,index)
    imgContours.append(imgSymbol)
    w=Xmax-Xmin
    h=Ymax-Ymin
    area=w*h
    sumBlack=sum(sum(1-imgSymbol))
    sumWhite=sum(sum(imgSymbol))
    whiteRatio= sumWhite/area
    blackRatio= sumBlack/area
    if(area<100 or (w<12 and w/h<0.5) or (h<12 and h/w<0.5) or whiteRatio>0.8 or blackRatio>=0.9):
        continue
    cv2.imwrite("contours/"+str(index)+".png",imgSymbol*255)
    index+=1
imgContours=np.array(imgContours)


In [916]:

# Load picture, convert to grayscale and detect edges 

#another function


# image_rgb = data.coffee()[0:220, 160:420] 
# image_gray = color.rgb2gray(imgSymbol) 
# edges = canny(image_gray, sigma=2.0, low_threshold=0.55, high_threshold=0.8) 
# cv2.imwrite("e.png",edges*255)
# result = hough_ellipse(edges, accuracy=20, threshold=250, min_size=100, max_size=120) 
# if(len(result)):
#     print("P")
#     result.sort(order='accumulator') 
#     best = list(result[-1]) 
#     yc, xc, a, b = [int(round(x)) for x in best[1:5]] 
#     orientation = best[5]
#     cy, cx = ellipse_perimeter(yc, xc, a, b, orientation) 
#     image_rgb[cy, cx] = (0, 0, 255) 
#     edges = color.gray2rgb(img_as_ubyte(edges)) 
#     edges[cy, cx] = (250, 0, 0)
#     fig2, (ax1, ax2) = plt.subplots(ncols=2, nrows=1, figsize=(8, 4), sharex=True, sharey=True, subplot_kw={'adjustable':'box-forced'}) 
#     ax1.set_title('Original picture') 
#     ax1.imshow(image_rgb) 
#     ax2.set_title('Edge (white) and result (red)') 
#     ax2.imshow(edges) 
#     plt.show() 